## Web Scraping of the Box Offices Mojo Database provided from IMdB

https://www.boxofficemojo.com/year/world/2019/

Import the libraries:

In [44]:
import pandas as pd
import numpy as np
import re
import datetime
import requests
import bs4

pd.set_option('display.max_rows', 50)


Base and variable URL:

In [45]:
base2 = "https://www.boxofficemojo.com/year/world/"
base = "https://www.boxofficemojo.com/year/"
sufix = "/?grossesOption=totalGrosses"
years = [str(i)+"/" for i in range(2000,2020)]

Empty DataFrame:

In [46]:
df = pd.DataFrame(columns=["year","release", "gross" , "max_th", "opening", "open_th", "open", "close", "distributor"])


Scrap the data for the last 20 years (2018 - 2019):

In [47]:
# problematic stucture when there is no distributor, small function to solve it
def distributor(movies):
    if movies.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal") == None:
        return("-")
    else:
        return movies.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal").contents[0] 


In [48]:
# empty dataframe
df = pd.DataFrame(columns=["year","rank","release", "gross" , "max_th", "opening", "open_th", "open", "close", "distributor"])

# loop for fetch the data
for year in years:
    url = base + year + sufix
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    movies = soup.find("div", class_="a-section imdb-scroll-table-inner").find_all("tr")
    for movie in movies[1:]:
        rank = str(movie.find_all("td", class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column")[0].contents[0])
        release = str(movie.find("a", class_="a-link-normal").contents[0])
        gross = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[0].contents[0].replace(",","").replace("$","").replace("-","0")))
        max_th = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer")[0].contents[0].replace(",","").replace("-","0")))
        opening = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-money")[0].contents[0].replace(",","").replace("$","").replace("-","0")))
        open_th = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer")[1].contents[0].replace(",","").replace("-","0")))
        open_ = str(movie.find_all("td", class_="a-text-left mojo-field-type-date a-nowrap")[0].contents[0])
        close_ = str(movie.find_all("td", class_="a-text-left mojo-field-type-date a-nowrap")[1].contents[0])
        distributor = str("-" if movie.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal") == None else movie.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal").contents[0])
                                            
        df = df.append({"year":int(year[:-1]), "rank":rank, "release":release, "gross":gross, "max_th":max_th, "opening":opening, "open_th":open_th, "open":open_, "close":close_, "distributor":distributor}, ignore_index=True)
     

In [43]:
df

,year,rank,release,gross,max_th,opening,open_th,open,close,distributor
0,2019,1,Avengers: Endgame,858373000,4662,357115007,4662,Apr 26,Sep 12,Walt Disney Studios Motion Pictures
1,2019,2,The Lion King,543638043,4802,191770759,4725,Jul 19,Dec 5,Walt Disney Studios Motion Pictures
2,2019,3,Star Wars: Episode IX - The Rise of Skywalker,503863021,4406,177383864,4406,Dec 20,-,Walt Disney Studios Motion Pictures
3,2019,4,Frozen II,470637370,4440,130263358,4440,Nov 22,-,Walt Disney Studios Motion Pictures
4,2019,5,Toy Story 4,434038008,4575,120908065,4575,Jun 21,Dec 5,Walt Disney Studios Motion Pictures
...,...,...,...,...,...,...,...,...,...,...
783,2019,784,Game Day,1624,4,1624,4,Oct 4,-,Ammo Content
784,2019,785,American Playhouse: The Killing Floor,1583,1,1583,1,Nov 22,-,Film Movement
785,2019,786,The Hours and Times,1273,1,1273,1,Mar 1,-,Oscilloscope
786,2019,787,The Untold Story,790,1,790,1,Jan 18,-,Ammo Content


It works! :)

In [13]:
df.to_csv("domestic_bo.csv")